In [13]:
import cv2
import cv2.aruco as aruco
import re
import numpy as np

In [14]:
def get_aruco_dict_from_filename(filename):
    match = re.search(r'(\d+)x(\d+)_(\d+)', filename)
    if not match:
        raise ValueError("Invalid filename format")

    bits = int(match.group(1))
    total = int(match.group(3))

    key = f'DICT_{bits}X{bits}_{total}'
    return getattr(aruco, key)

In [15]:
def quad_size(quad):
    quad = quad.reshape(4,2)
    return min(
        np.linalg.norm(quad[0]-quad[1]),
        np.linalg.norm(quad[1]-quad[2])
    )

def enhance_for_large_markers(gray):
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
    eq = clahe.apply(gray)

    blur = cv2.GaussianBlur(eq, (5,5), 0)
    sharp = cv2.addWeighted(eq, 1.15, blur, -0.15, 0)

    return sharp


In [20]:



filename = "ArUco_videos/Video7(5x5_250).mp4"
cap = cv2.VideoCapture(filename)
get_aruco_dict_from_filename(filename)

dict_id = get_aruco_dict_from_filename(filename)
dictionary = aruco.getPredefinedDictionary(dict_id)
parameters = aruco.DetectorParameters()
detector = aruco.ArucoDetector(dictionary, parameters)

print("Video opened:", cap.isOpened())

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame2 = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 1. Mild contrast normalization
    clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
    gray_eq = clahe.apply(gray)

    # 2. Edge-preserving denoising (very mild)
    denoised = cv2.bilateralFilter(gray_eq, d=7, sigmaColor=50, sigmaSpace=50)

    # 3. Mild unsharp masking (geometry-preserving)
    blur = cv2.GaussianBlur(denoised, (5,5), 0)
    sharp = cv2.addWeighted(denoised, 1.2, blur, -0.2, 0)
    

    corners, ids, rejected = detector.detectMarkers(gray)
    small_candidates = any(quad_size(q) < 25 for q in rejected)
    if ids is None and not small_candidates:
        processed = enhance_for_large_markers(gray)
        corners, ids, rejected = detector.detectMarkers(processed)
        aruco.drawDetectedMarkers(frame, corners, ids)
    else:
        
        corners, ids, rejected = detector.detectMarkers(gray)
        aruco.drawDetectedMarkers(frame, corners, ids)


    # corners2, ids2, rejected2 = detector.detectMarkers(blur)
    # corners3, ids3, rejected3 = detector.detectMarkers(sharp)
    # # corners4, ids4, rejected4 = detector.detectMarkers(blur2)
    # # corners5, ids5, rejected5 = detector.detectMarkers(unsharp2)
    # if ids is not None:
    #     aruco.drawDetectedMarkers(frame, corners, ids)
    #     aruco.drawDetectedMarkers(blur, corners2, ids2)
    #     aruco.drawDetectedMarkers(sharp, corners3, ids3)
        # aruco.drawDetectedMarkers(blur2, corners4, ids4)
        # aruco.drawDetectedMarkers(frame2, corners5, ids5)
    cv2.imshow("ArUco Detection", frame)
    # cv2.imshow("blur", blur)
    # cv2.imshow("unsharp", sharp)
    # cv2.imshow("blur2", blur2)
    # cv2.imshow("unsharp2", frame2)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Video opened: True


In [17]:
# # Core imports
# import cv2
# import cv2.aruco as aruco
# import re
# import numpy as np
# import matplotlib.pyplot as plt
# from IPython.display import clear_output
# plt.rcParams['figure.figsize'] = (12,6)

# # Helper functions: parsing, filters, detection, visualization
# def get_aruco_dict_from_filename(filename):
#     # expect patterns like '4x4_250' inside filename
#     match = re.search(r'(\d+)x(\d+)_(\d+)', filename)
#     if not match:
#         raise ValueError('Invalid filename format: expected e.g. 4x4_250')
#     bits = int(match.group(1))
#     total = int(match.group(3))
#     key = f'DICT_{bits}X{bits}_{total}'
#     if not hasattr(aruco, key):
#         raise ValueError('ArUco dictionary not found: ' + key)
#     return getattr(aruco, key)

# def unsharp_mask(gray, kernel_size=(5,5), sigma=1.0, amount=1.5):
#     # classic unsharp: sharpen = (1+amount)*orig - amount*blur
#     blur = cv2.GaussianBlur(gray, kernel_size, sigma)
#     sharpen = cv2.addWeighted(gray, 1+amount, blur, -amount, 0)
#     return np.clip(sharpen, 0, 255).astype(np.uint8)

# def laplacian_sharpen(gray, k=1.0):
#     lap = cv2.Laplacian(gray, cv2.CV_16S, ksize=3)
#     lap = cv2.convertScaleAbs(lap)
#     sharpen = cv2.addWeighted(gray, 1.0, lap, 0.7, 0)
#     return np.clip(sharpen,0,255).astype(np.uint8)

# def apply_filters(gray):
#     # produce a set of candidate images for detection comparison
#     results = {}
#     results['orig'] = gray
#     results['mean'] = cv2.blur(gray, (5,5))
#     results['gauss'] = cv2.GaussianBlur(gray, (5,5), 0)
#     results['median'] = cv2.medianBlur(gray, 5)
#     results['bilateral'] = cv2.bilateralFilter(gray, 9, 75, 75)
#     results['clahe'] = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(gray)
#     results['unsharp'] = unsharp_mask(results['clahe'], kernel_size=(5,5), sigma=1.0, amount=1.2)
#     results['lap_sharp'] = laplacian_sharpen(results['clahe'])
#     results['canny'] = cv2.Canny(results['gauss'], 50, 150)
#     return results

# def detect_and_visualize(frame, dictionary, parameters=None, draw_on=None):
#     # Returns annotated output and detection meta for a BGR frame
#     if parameters is None:
#         parameters = aruco.DetectorParameters()
#     detector = aruco.ArucoDetector(dictionary, parameters)
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     candidates = apply_filters(gray)
#     detections = {}
#     for name, img in candidates.items():
#         # detector.detectMarkers expects grayscale images
#         try:
#             corners, ids, rejected = detector.detectMarkers(img)
#         except Exception:
#             # fallback: try converting to uint8 if needed
#             corners, ids, rejected = detector.detectMarkers(img.astype(np.uint8))
#         detections[name] = (corners, ids)
#     # Build a color annotation of the original frame showing best guess(s)
#     annotated = frame.copy() if draw_on is None else draw_on
#     # Draw detections for selected stages to compare: orig, clahe, unsharp, lap_sharp
#     for stage in ['orig','clahe','unsharp','lap_sharp']:
#         corners_ids = detections.get(stage)
#         if corners_ids is None:
#             continue
#         corners, ids = corners_ids
#         if ids is not None:
#             # use different colors per stage (visual comparison)
#             color = (0,255,0) if stage=='orig' else ((255,0,0) if stage=='clahe' else (0,0,255))
#             aruco.drawDetectedMarkers(annotated, corners, ids, borderColor=color)
#     return annotated, candidates, detections

# def show_comparison_grid(orig_bgr, candidates, annotated=None):
#     # show: orig color, mean, gauss, clahe, unsharp, lap_sharp, canny, annotated
#     ncols = 4
#     imgs = []
#     imgs.append((cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2RGB),'original'))
#     order = ['mean','gauss','clahe','unsharp','lap_sharp','canny']
#     for k in order:
#         img = candidates.get(k)
#         if img is None:
#             imgs.append((np.zeros_like(cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2GRAY)),'') )
#         else:
#             if img.ndim==2:
#                 imgs.append((cv2.cvtColor(img, cv2.COLOR_GRAY2RGB), k))
#             else:
#                 imgs.append((img, k))
#     if annotated is not None:
#         imgs.append((cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB),'annotated'))
#     total = len(imgs)
#     rows = (total + ncols - 1)//ncols
#     fig, axes = plt.subplots(rows, ncols, figsize=(4*ncols,3*rows))
#     axes = axes.flatten() if hasattr(axes, 'flatten') else [axes]
#     for ax in axes:
#         ax.axis('off')
#     for i, (img, title) in enumerate(imgs):
#         axes[i].imshow(img, cmap='gray')
#         axes[i].set_title(title)
#     plt.tight_layout()
#     plt.show()

# # Main demo: process a video and display comparisons for successive frames
# filename = 'ArUco_videos/Video9(4x4_250).mp4'
# cap = cv2.VideoCapture(filename)
# if not cap.isOpened():
#     raise RuntimeError('Failed to open video: ' + filename)

# dict_id = get_aruco_dict_from_filename(filename)
# dictionary = aruco.getPredefinedDictionary(dict_id)
# parameters = aruco.DetectorParameters()
# # Tune detector parameters here if needed, e.g.:
# parameters.adaptiveThreshConstant = 7
# parameters.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX if hasattr(aruco, 'CORNER_REFINE_SUBPIX') else 1

# frame_idx = 0
# max_frames = 200
# # we'll show one frame every `step` frames to keep notebook responsive
# step = 10
# while frame_idx < max_frames:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     if frame_idx % step != 0:
#         frame_idx += 1
#         continue
#     annotated, candidates, detections = detect_and_visualize(frame, dictionary, parameters)
#     clear_output(wait=True)
#     print(f'Frame {frame_idx}')
#     show_comparison_grid(frame, candidates, annotated=annotated)
#     frame_idx += 1
# cap.release()
# print('Done')